In [22]:
from IPython.display import Markdown as md
from IPython.display import display_markdown as dm
import numpy as np
from numpy.linalg import norm
# import pandas as pd
import matplotlib.pyplot as plt
import sympy as smp
import scipy as scp
from scipy.interpolate import interp1d

In [ ]:
class UAV:
    rho_0 = 1.225
    rho = rho_0
    g = 9.81
    mu = 0.0289644
    R = 8.3144598
    T = 273 + 15
    def __init__(self, airfoil, S, mass, V, TR, AR, FusWidth, cr=0, Lt = 0, H = 0):
        self.mass = mass
        self.S = S
        self.d = FusWidth * 1.2
        if cr == 0:
            self.AR = AR
            self.TR = TR
            self.c = np.sqrt(self.S/self.AR)
            self.b = self.S/self.c
            self.cr = self.S / (self.d + (1 + self.TR)*(self.b - self.d)/2)
            self.ct = self.cr * self.TR
        else:
            self.AR = AR
            self.cr = cr
            self.c = np.sqrt(self.S/self.AR)
            self.b = self.S/self.c
            self.ct = 2*(self.S - self.cr*self.d)/(self.b - self.d) - self.cr
            self.TR = self.ct/self.cr
            if (self.TR >= 1):
                self.TR = 1
                self.c = np.sqrt(self.S/self.AR)
                self.ct = self.c
                self.cr = self.ct

        ex = np.array([1.,0])
        ey = np.array([0,1.])
        a = 3/4*self.cr * ex + ey * 0.5 * (self.b - self.d) + ex*1/4*self.ct - ex*self.cr
        b = 3/4*self.cr * ex + ey * 0.5 * (self.b - self.d) - ex * 3/4*self.ct
        e = b + ex*0.5*self.ct
 
        self.SweepLE = np.acos(np.dot(ey, a) / (norm(a) * norm(ey)))
        self.SweepTE = np.acos(np.dot(ey, b) / (norm(b) * norm(ey)))

        self.SweepHalf = np.acos(np.dot(ey, e - ex*0.5*self.cr) / (norm(e - ex*0.5*self.cr) * norm(ey)))
        self.SweepQuart = 0
        self.Lt = Lt
        self.Va = V
        self.H = H
        self.airfoilName = airfoil
        self.airfoilDat = np.loadtxt('{}.txt'.format(airfoil), skiprows = 11, dtype=float)

        self.__GetAirDensity(self.H)

    def __GetAirDensity(self, h):
        self.rho = self.rho_0*np.exp(-self.g * self.mu * h / (self.R * self.T))
    
    def GetAirfoilCl(self, a):
        alpha = self.airfoilDat[:, 0]
        Cl = self.airfoilDat[:, 1]
        fCl = interp1d(alpha, Cl)
        return fCl(a)
    
    def GetAirfoilCd(self, a):
        alpha = self.airfoilDat[:, 0]
        Cd = self.airfoilDat[:, 2]
        fCd = interp1d(alpha, Cd)
        return fCd(a)
    
    def GetAirfoilCm(self, a):
        alpha = self.airfoilDat[:, 0]
        Cm = self.airfoilDat[:, 4]
        fCm = interp1d(alpha, Cm)
        return fCm(a)
    
    def GetAirfoilClmax(self):
        curr = 0
        a=0
        alpha = self.airfoilDat[:, 0]
        Cl = self.airfoilDat[:, 1]
        fCl = interp1d(alpha, Cl)
        for el in np.linspace(0, alpha[-1], 1000):
            if fCl(el) >= curr:
                curr = fCl(el)
                a = el
        return (a, curr)
    
    def GetAirfoilClslope(self, a):
        alpha = self.airfoilDat[:, 0]
        Cl = self.airfoilDat[:, 1]
        fCl = interp1d(alpha, Cl)
        return (-fCl(a) + fCl(a + 0.5))/0.5 *180/np.pi
    
    def GetCLmax(self):
        return 0.9 * np.cos(self.SweepQuart) * self.GetAirfoilClmax()[1]
    
    def GetAirfoilCl0(self):
        alpha = self.airfoilDat[:, 0]
        Cl = self.airfoilDat[:, 1]
        fCl = interp1d(alpha, Cl)
        for el in np.linspace(-4, 0, 1000):
            if abs(fCl(el)) < 0.01:
                return el

    def GetCL0(self):
         az = (-1)*self.GetAirfoilCl0()/self.GetAirfoilClslope(0)
         return abs(az)*self.GetCLslope()


    def GetCruiseSpeed(self):
        return np.sqrt(2 * self.g * self.mass / (self.rho * self.S * 0.4))
    
    def GetDynamicPressure(self):
        self.__GetAirDensity(self.H)
        if self.Va != 0:
            return self.rho * self.Va**2 / 2
        else:
            return self.rho * self.GetCruiseSpeed()**2 / 2
    
    def GetSpeedSound(self):
        return np.sqrt(1.4 * self.R * self.T/self.mu)
    
    def GetCLslope(self):
        """Расчитывает наклон кривой коэффициента подъемной силы
        для секции крыла в 1/rad"""
        beta = 1 - (self.Va/self.GetSpeedSound())**2
        k = self.GetAirfoilClslope(0) / (2*np.pi)
        x = self.AR/k * (beta**2 + np.tan(self.SweepHalf)**2)**0.5
        eta = 0.98
        return 2*np.pi*self.AR / (2 + np.sqrt(x**2 + 4))*eta


p = UAV('SD7032', 0.366, 3.5, 18, 0.5, 7, 0.2, cr=0.25)
p.b, p.cr, p.c, p.ct, p.TR, p.AR, p.SweepHalf, p.GetCLmax(), p.GetCLslope(), p.GetAirfoilClslope(2), p.GetCL0()

(np.float64(1.600624877977348),
 0.25,
 np.float64(0.22866069685390683),
 np.float64(0.19979333386125891),
 np.float64(0.7991733354450357),
 7,
 np.float64(0.018447763001596743),
 np.float64(1.2894045945945947),
 np.float64(4.59593029057681),
 np.float64(6.061893472484117),
 np.float64(2.8393533541652167))

### Модель коэффициента подъемной силы
$$C_L = C_{L_{0}} + C_{L_{\alpha}}\alpha + C_{L_{\delta_{e}}}\delta_{e} + C_{L_{q}}\frac{c}{2v_a}q$$

#### Наклон кривой коэффициента подъемной силы $C_{L_{\alpha}}$
![.](LiftCurveSlope.jpg)
![.](LiftCurveSlopeRaymer.jpg)

#### Угол атаки нулевой подъемной силы

![.](ZeroLiftCoef.jpg)

#### Коэффициент эффективности Освальда

![.](OsvaldEfficiency.jpg)

### Модель коэффициента лобового сопротивления
$$C_D = C_{D_{0}} + C_{D{\alpha}}\alpha + C_{D_{\delta_{e}}}\delta_{e} + C_{D_{q}}\frac{c}{2v_a}q$$
Для более точной оценки лобового сопротивления в зависимости от угла атаки и геометрических характеристик ЛА используем дополненую квадратичную модель коэффициента лобового сопроивления
$$C_{D_{0}} + C_{D_{\alpha}}\alpha = C_{D}(\alpha) = C_{D_{0}} + C_{D_{i}} = C_{D_{0}} + \frac{(C_{L}(\alpha) - C_{LmD})^{2}}{\pi e AR}$$
где $C_{D_{0}}$ - паразитное сопроивление, $C_{D_{i}}$ - индуцированное сопротивление засчет подъемной силы, $e$ - коэффициент эффективности Освальда
$$C_{D_{\alpha}} = \frac{\partial C_{D}(\alpha)}{\partial \alpha} = \frac{2}{\pi e AR}(C_{L_{0}} + C_{L_{\alpha}} - C_{LmD})C_{L_{\alpha}}$$

In [ ]:
m = 3.5 #масса ЛА
v_a = 18 #средняя ветровая скорость
C_L = 0.45 #коэффициент подъёмной силы для профиля SD7032

q = pho*(v_a**2)/2
S_w = m*g/(q*C_L)

dm(md(f"$ S_w = {round(S_w, 2)} $"))

$ S_w = 0.38 $

In [ ]:
#ИНИЦИАЛИЗАЦИЯ
C_R = 0.25 #корневая хорда крыла
C_e = 0.2 #концевая хорда крыла
TR_w = C_e/C_R #сужение крыла
AR_w = 7 #удлинение крыла
d = 0.2 # длина центроплана

#РАСЧЁТ
C_m = 0.5*(C_R+C_e) #полусумма оснований
b_w = AR_w*(C_m + np.sqrt(C_m**2 - 4*d*(C_m-C_R)/AR_w))/2 #размах крыла
C_w = b_w/AR_w #средняя хорда крыла (из b_w)

#ВЫВОД
dm(md(f"$b_w = {round(b_w, 2)}, C_w = {round(C_w, 2)}$"))

$b_w = 1.6, C_w = 0.23$

In [ ]:
#ИНИЦИАЛИЗАЦИЯ
c_HT = 0.5
c_VT = 0.02
L = 0.525

#РАСЧЁТ
S_VT = 0.5*c_VT*b_w*S_w/L
TS_HT = c_HT*C_w*S_w/L 

#ВЫВОД
dm(md(f"$S_{{VT}} =  {round(S_VT,3)}$"))
dm(md(f"$S_{{HT}} =  {round(S_HT,3)}$"))

$S_{VT} =  0.012$

NameError: name 'S_HT' is not defined

In [ ]:
#ИНИЦИАЛИЗАЦИЯ
AR_t = 3 #удлинение хвостового оперения

#РАСЧЁТ
#средние хорды вертикального и горизонтального оперений
C_HT = np.sqrt(S_HT/AR_t)
C_VT = np.sqrt(S_VT/AR_t)
#размахи вертикального и горизонтального оперений
b_HT = S_HT/C_HT
b_VT = S_VT/C_VT
#ВЫВОД
dm(md(f"$C_{{VT}} =  {round(C_VT,3)}$"))
dm(md(f"$C_{{HT}} =  {round(C_HT,3)}$"))
dm(md(f"$b_{{VT}} =  {round(b_VT,3)}$"))
dm(md(f"$b_{{HT}} =  {round(b_HT,3)}$"))
# print("b_VT = ", round(b_VT,3))
# print("C_HT = ", round(C_HT,3))
# print("b_HT = ", round(b_HT,3))

$C_{VT} =  0.062$

$C_{HT} =  0.167$

$b_{VT} =  0.187$

$b_{HT} =  0.5$

In [ ]:
#ИНИЦИАЛИЗАЦИЯ

#РАСЧЁТ
#угол наклона лопатки относительно горизонтальной плоскости
theta = np.arctan(b_VT/b_HT)
S_T = S_HT/np.cos(theta) #площадь лопатки V_tail

#ВЫВОД
dm(md(f"$\\theta = {round(theta*360/(2*np.pi), 2)}\\degree$"))
dm(md(f"$S_T = {round(S_T, 3)}$"))

$\theta = 20.51\degree$

$S_T = 0.089$

In [ ]:
#ИНИЦИАЛИЗАЦИЯ
TR_t = 0.5 #сужение для хвостового оперения
b_T = b_HT/2 #длина лопатки

#РАСЧЁТ
#средняя, корневая и концевая хорды лопаток V-tail
c = np.sqrt(S_T/AR_t)
c_r = c/0.75
c_e = 0.5*c_r

#ВЫВОД
dm(md(f"$c = {round(c, 3)}$"))
dm(md(f"$c_r = {round(c_r, 3)}$"))
dm(md(f"$c_e = {round(c_e, 3)}$"))

$c = 0.172$

$c_r = 0.23$

$c_e = 0.115$

In [ ]:
df = pd.read_csv('SD7032 C_L.csv', delimiter = '\t')
alpha = df['alpha'] #угол атаки в градусах
CL = df['CL']
start = 13
stop = 67
t_approx = np.polyfit(alpha[start:stop], CL[start:stop], 1)
f_approx = np.poly1d(t_approx)[1]*360/(2*np.pi) #тангенс угла наклона линейной части поляры крыла C_l(alpha)
C_L_alpha = 0.9*f_approx #коэффициент при первой степени разложения коэффициента подъёмной силы по альфа  
dm(md(f"$C_{{L\\alpha}} =  {round(C_L_alpha,3)} , SD7032$"))


$C_{L\alpha} =  5.463 , SD7032$

In [ ]:
df1 = pd.read_csv('xf-n0009sm-il-500000.csv', delimiter = ',')
alpha = df1['Alpha'] #угол атаки в градусах
CL = df1['Cl']
start = 40
stop = 68
t_approx = np.polyfit(alpha[start:stop], CL[start:stop], 1)
f_approx = np.poly1d(t_approx)[1]*360/(2*np.pi) #тангенс угла наклона линейной части поляры крыла C_l(alpha)
C_L_alpha1 = 0.9*f_approx #коэффициент при первой степени разложения коэффициента подъёмной силы по альфа  
dm(md(f"$C_{{L\\alpha}} =  {round(C_L_alpha1,3)}, NACA0009$"))

$C_{L\alpha} =  7.428, NACA0009$

In [ ]:
M = q*S_w*C_L
C1 = C_L_alpha1
varphi = np.sqrt(M/(q*L*S_w*C1*2)) #угол установки
X_cp = S_T*C1*L*0.9/(S_w*C_L_alpha)
dm(md(f"$\\varphi = {round(varphi*360/(2*np.pi), 1)} \\degree$"))
dm(md(f"$X_{{c.p}} = {round(X_cp , 3)}$"))

$\varphi = 13.8 \degree$

$X_{c.p} = 0.149$

In [ ]:
#РЕЗУЛЬТАТЫ
dm(md(f"$ S_w = {round(S_w, 2)} $"))
dm(md(f"$b_w = {round(b_w, 2)}, C_w = {round(C_w, 2)}$"))
dm(md(f"$S_{{VT}} =  {round(S_VT,3)}$"))
dm(md(f"$S_{{HT}} =  {round(S_HT,3)}$"))
dm(md(f"$C_{{VT}} =  {round(C_VT,3)}$"))
dm(md(f"$C_{{HT}} =  {round(C_HT,3)}$"))
dm(md(f"$b_{{VT}} =  {round(b_VT,3)}$"))
dm(md(f"$b_{{HT}} =  {round(b_HT,3)}$"))
dm(md(f"$\\theta = {round(theta*360/(2*np.pi), 2)}\\degree$"))
dm(md(f"$S_T = {round(S_T, 3)}$"))
dm(md(f"$c = {round(c, 3)}$"))
dm(md(f"$c_r = {round(c_r, 3)}$"))
dm(md(f"$c_e = {round(c_e, 3)}$"))
dm(md(f"$C_{{L\\alpha}} =  {round(C_L_alpha,3)} , SD7032$"))
dm(md(f"$C_{{L\\alpha}} =  {round(C_L_alpha1,3)}, NACA0009$"))
dm(md(f"$X_{{c.p}} = {round(X_cp , 3)}$"))
dm(md(f"$\\varphi = {round(varphi*360/(2*np.pi), 1)} \\degree$"))

$ S_w = 0.38 $

$b_w = 1.6, C_w = 0.23$

$S_{VT} =  0.012$

$S_{HT} =  0.083$

$C_{VT} =  0.062$

...

$c_e = 0.115$

$C_{L\alpha} =  5.463 , SD7032$

$C_{L\alpha} =  7.428, NACA0009$

$X_{c.p} = 0.149$

$\varphi = 13.8 \degree$